# マルチエージェントシステムのアーキテクチャーを紐解く

## 事前準備

In [ ]:
%pip install --upgrade --user google-adk

In [19]:
# カーネル再起動
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

In [1]:
import copy, os, uuid
import vertexai

from google.genai.types import Part, Content
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.sequential_agent import SequentialAgent

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

/home/jupyter/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(


In [2]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'====\n{event}\n====')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

In [3]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
         return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
        
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

## リサーチエージェントの定義

In [4]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [5]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
前段のエージェントは、５項目程度の調査対象トピックを指定します。

* 出力形式
日本語で出力。
調査レポートは、トピックごとに客観的情報をまとめます。各トピックについて、５文以上の長さで記述すること。
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    instruction=instruction,
)

In [6]:
research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        copy.deepcopy(research_agent1),
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        copy.deepcopy(research_agent2),
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

## ライターエージェント、レビューエージェントの定義

In [7]:
instruction = '''
あなたの役割は、特定のテーマに関する気軽な読み物記事を書くことです。
記事の「テーマ」と、その内容に関連する「調査レポート」が与えられるので、
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

レビュアーによる修正ポイントが指摘された際は、直前に書いた記事を指摘に従って書き直してください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 記事タイトルは付けないで、次の構成で出力します。各セクションタイトルは、内容に合わせてアレンジしてください。
0. 導入：セクションタイトルを付けないで、この記事を読みたくなる導入を１〜２文にまとめます。
1. 概要：トピックの全体像をまとめて簡単に説明します。
2. 最新情報：特に注目したい新しい情報を取り上げます。
3. 実践：トピックに関して、読者自身がやってみるとよさそうな事を１つ紹介します。
4. まとめ

- 各セクションのタイトルはマークダウンヘッダ ## 付けます。必要に応じて箇条書きのマークダウンを使用します。
- それ以外のマークダウンによる装飾は行いません。
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [8]:
instruction = f'''
あなたの役割は、読み物記事をレビューして、記事の条件にあった内容にするための改善コメントを与える事です。

* 記事の条件
- 記事は、はじめに４０文字程度のタイトルがあること。
　今日から役立つ生活情報があって「すぐに読まなきゃ」と読者が感じるタイトルにすること。
　タイトルはマークダウンヘッダ # をつけること。
- タイトルの直後に「なぜいまこのテーマを取り上げるのか」をまとめた導入を加えて、読者にこの記事を読む動機づけを与えます。
- 各セクションのサブタイトルには、絵文字を用いて親しみやすさを出すこと。
- 読者が今日から実践できる具体例が３つ以上紹介されていること。

* 出力形式
- 日本語で出力。
- はじめに、記事の良い点を説明します。
- 次に、修正ポイントを箇条書きで出力します。
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    instruction=instruction,
)

In [9]:
write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        copy.deepcopy(writer_agent),
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        copy.deepcopy(review_agent),
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

## 業務フローエージェントの定義

In [10]:
root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        copy.deepcopy(research_agent),
        copy.deepcopy(write_and_review_agent),
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

## 実行例

In [11]:
client = LocalApp(root_agent)

DEBUG=False
query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！記事作成のお手伝いをします。

私ができることは以下の通りです。

1.  **記事テーマの調査:** 記事に必要な情報をリサーチエージェントが調査し、レポートを作成します。
2.  **記事の作成とレビュー:** ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。
3.  **記事の修正:** 修正が必要な場合も、ライターエージェントとレビューエージェントが対応します。

記事のテーマをご指定いただければ、すぐに記事作成に取り掛かります。まずはリサーチエージェントに調査レポートを依頼しますね。



In [12]:
query = f'''
「夏を乗り切る郷土料理のスパイシーアレンジ」をテーマに記事を作成してください。
'''
result = await client.stream(query)

承知いたしました。「夏を乗り切る郷土料理のスパイシーアレンジ」をテーマに記事を作成します。

まずは、リサーチエージェントに調査レポートを依頼します。少々お待ちください。



---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。

承知いたしました。「夏を乗り切る郷土料理のスパイシーアレンジ」に関する調査レポートを作成するために、以下のトピックを選定しました。このリストに基づいて、後続のエージェントが詳細な調査を行います。

---
### 調査対象トピックリスト

1.  **夏バテ対策としての郷土料理の役割と伝統**:
    *   日本の郷土料理が古くから夏に食べられてきた背景や知恵。
    *   夏バテのメカニズムと、郷土料理が持つ栄養学的・生理学的効果（消化促進、体温調節、疲労回復など）。
    *   地域ごとの代表的な夏向け郷土料理の紹介。

2.  **スパイシーアレンジの健康効果と食欲増進作用**:
    *   香辛料（唐辛子、山椒、胡椒、カレー粉など）が持つ具体的な健康効果（発汗作用、血行促進、食欲増進、消化促進など）。
    *   夏にスパイシーなものを食べることのメリットと、体が求める理由。

3.  **具体的な郷土料理のスパイシーアレンジ例とアイデア**:
    *   日本各地の郷土料理（例：冷や汁、ゴーヤチャンプルー、そうめん、鶏飯など）をスパイシーにアレンジする具体的なレシピや方法。
    *   使用する香辛料の種類とその組み合わせ、風味を損なわないための工夫。

4.  **スパイシーアレンジのコツと注意点**:
    *   辛さのレベル調整方法や、香辛料の選び方・使い方。
    *   郷土料理の伝統的な風味とスパイシーさのバランスを取るポイント。
    *   食べ過ぎや脱水など、健康面での注意喚起と対策。

5.  **郷土料理の新たな楽しみ方と食文化の提案**:
    *   伝統的な郷土料理に現代的なアレンジを加える意義と、食文化の進化。
    *   夏を健康的に、そして楽しく乗り切るための新しい食の提案。

---

## 選定したトピックに基づいて、調査レポートを作成します。

## 調査レポート：「夏を乗り切る郷土料

In [13]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

承知いたしました。それでは、ライターエージェントとレビューエージェントに記事の作成を依頼します。



---
## ライターエージェントが記事を執筆します。
---

今年の夏も暑そうですね。そんな日本の厳しい夏を乗り切るために、昔から受け継がれてきた知恵が詰まった「郷土料理」に、ピリッと刺激的な「スパイシーアレンジ」を加えてみませんか？

## 郷土料理と香辛料の素敵な出会い

日本の夏は、高温多湿で体力を消耗しやすい季節。食欲が落ちたり、体がだるくなったりする「夏バテ」に悩まされる方も多いのではないでしょうか。そんなとき、古くから私たちの祖先は、その土地ならではの食材や気候に合わせた郷土料理で、夏を乗り切ってきました。例えば、宮崎のさっぱり冷や汁や、沖縄のゴーヤチャンプルーのように、体に必要な栄養を補給し、食欲を刺激してくれる工夫が凝らされているんです。

そして、ここに加わるのが「香辛料」のパワー！唐辛子のカプサイシンは汗をかいて体温調節を助けたり、血行を良くしたり。山椒や胡椒は胃腸の働きを活発にして、食欲を増進させてくれる効果が期待できます。これらの香辛料が持つ独特の香りと辛味は、疲れた体を目覚めさせ、食欲がなくても「食べたい！」という気持ちにさせてくれるから不思議ですよね。郷土料理の持つ栄養と、香辛料の食欲増進効果が組み合わさることで、まさに夏バテ対策の最強タッグが生まれるわけです。

## 伝統と革新が織りなす新たな食の楽しみ

「郷土料理って、ちょっと昔っぽい？」なんて思う方もいるかもしれません。でも、現代の食卓に合わせてスパイシーにアレンジすることで、驚くほど新鮮な魅力が引き出されるんです。これは、伝統的な食文化が時代とともに進化する、まさに「革新」の瞬間！

例えば、宮崎の冷や汁にラー油や豆板醤、チリパウダーを少し加えるだけで、深みと辛味が加わり、箸が止まらなくなります。沖縄のゴーヤチャンプルーにカレー粉を隠し味にしたり、生の唐辛子を効かせたりするのもおすすめです。定番のそうめんも、コチュジャンやエスニックなタレを加えれば、まるでアジアンスタイルの麺料理に変身。鹿児島名物の鶏飯には、柚子胡椒や七味唐辛子を添えると、あっさりとした出汁の旨味にピリッとしたアクセントが加わり、奥行きのある味わいになります。

このように、昔ながらの郷土料理に現代的なス

In [14]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)

承知いたしました。レビューエージェントからの指摘に基づき、記事の修正をライターエージェントに依頼します。



---
## ライターエージェントが記事を執筆します。
---

# 夏バテ知らず！ピリ辛郷土料理で元気チャージ術🔥

今年の夏も暑そうですね。そんな日本の厳しい夏を乗り切るために、昔から受け継がれてきた知恵が詰まった「郷土料理」に、ピリッと刺激的な「スパイシーアレンジ」を加えてみませんか？この記事では、その魅力をたっぷりお伝えします！

## 郷土料理と香辛料の素敵な出会い🌶️

日本の夏は、高温多湿で体力を消耗しやすい季節。食欲が落ちたり、体がだるくなったりする「夏バテ」に悩まされる方も多いのではないでしょうか。そんなとき、古くから私たちの祖先は、その土地ならではの食材や気候に合わせた郷土料理で、夏を乗り切ってきました。例えば、宮崎のさっぱり冷や汁や、沖縄のゴーヤチャンプルーのように、体に必要な栄養を補給し、食欲を刺激してくれる工夫が凝らされているんです。

そして、ここに加わるのが「香辛料」のパワー！唐辛子のカプサイシンは汗をかいて体温調節を助けたり、血行を良くしたり。山椒や胡椒は胃腸の働きを活発にして、食欲を増進させてくれる効果が期待できます。これらの香辛料が持つ独特の香りと辛味は、疲れた体を目覚めさせ、食欲がなくても「食べたい！」という気持ちにさせてくれるから不思議ですよね。郷土料理の持つ栄養と、香辛料の食欲増進効果が組み合わさることで、まさに夏バテ対策の最強タッグが生まれるわけです。

## 伝統と革新が織りなす新たな食の楽しみ✨

「郷土料理って、ちょっと昔っぽい？」なんて思う方もいるかもしれません。でも、現代の食卓に合わせてスパイシーにアレンジすることで、驚くほど新鮮な魅力が引き出されるんです。これは、伝統的な食文化が時代とともに進化する、まさに「革新」の瞬間！

このように、昔ながらの郷土料理に現代的なスパイスの知恵をプラスすることは、単なる味の変化にとどまりません。これまで郷土料理に馴染みがなかった若い世代や、海外の方々にも、日本の食文化の奥深さを知ってもらう良いきっかけにもなりますよね。食卓がもっと楽しく、もっと豊かになる新しい食の提案なんです。

## 今すぐ試せる！スパイシー郷土料理の楽しみ方🥢

さあ、あなたも郷土料理のスパイシ

In [15]:
query = f'''
これで完成です。完成した記事をもう一度表示して。
'''
result = await client.stream(query)

# 夏バテ知らず！ピリ辛郷土料理で元気チャージ術🔥

今年の夏も暑そうですね。そんな日本の厳しい夏を乗り切るために、昔から受け継がれてきた知恵が詰まった「郷土料理」に、ピリッと刺激的な「スパイシーアレンジ」を加えてみませんか？この記事では、その魅力をたっぷりお伝えします！

## 郷土料理と香辛料の素敵な出会い🌶️

日本の夏は、高温多湿で体力を消耗しやすい季節。食欲が落ちたり、体がだるくなったりする「夏バテ」に悩まされる方も多いのではないでしょうか。そんなとき、古くから私たちの祖先は、その土地ならではの食材や気候に合わせた郷土料理で、夏を乗り切ってきました。例えば、宮崎のさっぱり冷や汁や、沖縄のゴーヤチャンプルーのように、体に必要な栄養を補給し、食欲を刺激してくれる工夫が凝らされているんです。

そして、ここに加わるのが「香辛料」のパワー！唐辛子のカプサイシンは汗をかいて体温調節を助けたり、血行を良くしたり。山椒や胡椒は胃腸の働きを活発にして、食欲を増進させてくれる効果が期待できます。これらの香辛料が持つ独特の香りと辛味は、疲れた体を目覚めさせ、食欲がなくても「食べたい！」という気持ちにさせてくれるから不思議ですよね。郷土料理の持つ栄養と、香辛料の食欲増進効果が組み合わさることで、まさに夏バテ対策の最強タッグが生まれるわけです。

## 伝統と革新が織りなす新たな食の楽しみ✨

「郷土料理って、ちょっと昔っぽい？」なんて思う方もいるかもしれません。でも、現代の食卓に合わせてスパイシーにアレンジすることで、驚くほど新鮮な魅力が引き出されるんです。これは、伝統的な食文化が時代とともに進化する、まさに「革新」の瞬間！

このように、昔ながらの郷土料理に現代的なスパイスの知恵をプラスすることは、単なる味の変化にとどまりません。これまで郷土料理に馴染みがなかった若い世代や、海外の方々にも、日本の食文化の奥深さを知ってもらう良いきっかけにもなりますよね。食卓がもっと楽しく、もっと豊かになる新しい食の提案なんです。

## 今すぐ試せる！スパイシー郷土料理の楽しみ方🥢

さあ、あなたも郷土料理のスパイシーアレンジに挑戦してみませんか？いくつか試しやすいヒントをお伝えしますね。

### まずはここから！ピリ辛アレンジレシピのヒント

*   **宮崎の冷や汁**: ラー油や豆板